# Dmitriev Egor
e.dmitriev@innopolis.university | BS20-RO

In [1]:
from __future__ import unicode_literals, print_function, division
import pandas as pd

In [2]:
import sys
# setting path so we cam import src
sys.path.append('..')
from src.data.data import *
from src.models.models import *

In [3]:
# Load preporcessed dataset
df = pd.read_csv('../data/interim/preprocessed.tsv', sep='\t', index_col=0)
sentence_pairs = df[['reference', 'translation']].values

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [5]:
MAX_LENGTH = 64
# MAX_LENGTH = len(max(sentence_pairs.ravel(), key=len))
print(f'{MAX_LENGTH=}')

MAX_LENGTH=64


In [6]:
lang1, lang2 = Lang('tox'), Lang('detox')
input_lang, output_lang, pairs = prepareData(lang1, lang2, sentence_pairs, MAX_LENGTH)
print(random.choice(pairs))

Read 190358 sentence pairs
Trimmed to 13559 sentence pairs
Counting words...


Counted words:
tox 7642
detox 7888
['you re three grown men who never got to play baseball because you were weird and smelled and sat when you peed'
 'you re three grown men who never got a chance to play because you were weird and you got there to pee']


In [32]:
print(pairs[:3])

[array(['i m famous and you re dead', 'i m famous and you re done'],
      dtype=object), array(['the guy is totally irresponsible',
       'he s completely irresponsible'], dtype=object), array(['damn i m glad you re back in town yeah me too',
       'i m damn glad you re back in town'], dtype=object)]


In [7]:
hidden_size = 128
batch_size = 128

input_lang, output_lang, train_dataloader = get_dataloader(batch_size, sentence_pairs, lang1, lang2, MAX_LENGTH, device)

encoder = EncoderRNN(input_lang.n_words, hidden_size).to(device)
decoder = AttnDecoderRNN(hidden_size, output_lang.n_words, device=device, SOS_token=SOS_token, max_length=MAX_LENGTH).to(device)

train(train_dataloader, encoder, decoder, 80, print_every=5, plot_every=5, saveDir = '../models')

Read 190358 sentence pairs
Trimmed to 13559 sentence pairs
Counting words...
Counted words:
tox 7642
detox 7888


/home/genom10/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


0m 53s (- 13m 19s) (5 6%) 0.8363
[Errno 17] File exists: '../models'
1m 47s (- 12m 30s) (10 12%) 0.5329
[Errno 17] File exists: '../models'
2m 41s (- 11m 38s) (15 18%) 0.4320
[Errno 17] File exists: '../models'
3m 34s (- 10m 44s) (20 25%) 0.3540
[Errno 17] File exists: '../models'
4m 28s (- 9m 51s) (25 31%) 0.2908
[Errno 17] File exists: '../models'
5m 22s (- 8m 58s) (30 37%) 0.2386
[Errno 17] File exists: '../models'
6m 17s (- 8m 4s) (35 43%) 0.1970
[Errno 17] File exists: '../models'
7m 10s (- 7m 10s) (40 50%) 0.1642
[Errno 17] File exists: '../models'
8m 3s (- 6m 16s) (45 56%) 0.1381
[Errno 17] File exists: '../models'
8m 56s (- 5m 21s) (50 62%) 0.1181
[Errno 17] File exists: '../models'
9m 44s (- 4m 25s) (55 68%) 0.1015
[Errno 17] File exists: '../models'
10m 32s (- 3m 30s) (60 75%) 0.0888
[Errno 17] File exists: '../models'
11m 21s (- 2m 37s) (65 81%) 0.0778
[Errno 17] File exists: '../models'
12m 11s (- 1m 44s) (70 87%) 0.0687
[Errno 17] File exists: '../models'
13m 1s (- 0m 52s)

In [14]:
def save2file(file, path):
    filehandler = open(path, 'wb')
    pickle.dump(file, filehandler)

def loadFromFile(path):
    file_pi2 = open(path, 'rb') 
    file = pickle.load(file_pi2)
    return file

In [12]:
saveModels(encoder, decoder)
save2file(input_lang, '../models/lang_tox.obj')
save2file(output_lang, '../models/lang_detox.obj')

[Errno 17] File exists: '../models'


In [15]:
encoder = EncoderRNN(input_lang.n_words, hidden_size).to(device)
decoder = AttnDecoderRNN(hidden_size, output_lang.n_words, device=device, SOS_token=SOS_token, max_length=MAX_LENGTH).to(device)
encoder, decoder = loadModels(encoder, decoder, '../models/EncoderRNN_16.0.pt', '../models/AttnDecoderRNN_16.0.pt', )
input_lang = loadFromFile('../models/lang_tox.obj')
output_lang = loadFromFile('../models/lang_detox.obj')

In [31]:
evaluateRandomly(input_lang, output_lang, encoder, decoder, sentence_pairs, device)

> shot in the abdomen with a
= he was shot in the abdomen by a


TypeError: '_IncompatibleKeys' object is not callable